# Select GPU

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


# Import module 

In [6]:
import numpy as np
import pandas as pd
import math
import keras
import csv
import os
import random
import time
import pickle
import matplotlib
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Input,Conv1D, Dense, MaxPool1D, Activation, AvgPool1D,GlobalAveragePooling1D
from keras.layers import Flatten, Add, Concatenate, Dropout, BatchNormalization
from keras import regularizers, initializers
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error
from scipy.stats.stats import pearsonr
from keras.optimizers import Adam
from keras import regularizers
from data_tools import *
# from model_function import *

# Configuration

In [7]:
multi_CNN = False
activation = 'relu' # selu
use_meta = True

# Data import

In [16]:
'''read signal'''
file_path = '/data/put_data/medical/BG/1006_data/'
if multi_CNN:
    with open(file_path+'dict_X_C10_S5_50.pickle', 'rb') as handle:
        X50 = pickle.load(handle)
    with open(file_path+'dict_X_C10_S5_100.pickle', 'rb') as handle: # 100
        X5 = pickle.load(handle) 
    with open(file_path+'dict_X_C10_S5_200.pickle', 'rb') as handle: 
        X10 = pickle.load(handle) 
    with open(file_path+'dict_X_C10_S5_25.pickle', 'rb') as handle: 
        X25 = pickle.load(handle) 
else:
    with open(file_path+'dict_X_C10_S5_200.pickle', 'rb') as handle:
        X = pickle.load(handle)
'''read meta'''
with open(file_path+'dict_meta.pickle', 'rb') as handle:
    meta = pickle.load(handle)

In [29]:
with open(file_path+'dict_X_C10_all.pickle', 'rb') as handle:
    X = pickle.load(handle)

In [30]:
X['011_1'][0].shape

(10,)

In [28]:
X['011_1'][0].shape

(6000, 10)

# Parameter

In [13]:
# data split
split_ratio = 0.8

# model parameter
l_2 = 0
dropout_rate = 0.2
lr_rate = 1e-5

# Embedding
n_sample = 6000
n_meta = len(meta['003_1'])
n_channel = 10

# Convolution
kernel_size = 16
dense_layers = [100,70,1]
local_filters = 64
local_kernel_size = 3
local_pool_size = 2
filters = 64
strides = 1
layers = 10

# Training
batch_size = 64
n_batch_per_epoch = 200
epochs = 200
kernel_initializer = 'he_normal'

d = pd.read_csv("/data/put_data/medical/BG/new_IRB_summary_1119_1006.csv",dtype="str")
print(d[:5])

  Unnamed: 0 Person No Age Gender Height  SYS  DIA   HR    G  PWV   BMI  \
0          1         3  53      0    157  143  104   73  106  6.5  25.4   
1          2         3  53      0    157  144   95   73   95  6.5  25.4   
2          3         4  49      0    150  133   92   80   94    7  23.4   
3          4         4  49      0    150  117   85   75   99    7  23.4   
4          5         5  55      1  167.5  152   83  106   79    7  24.4   

  BP_drug DM DM_drug O_drug W_cir weight     ID      Date  Time  
0       0  0       0      1    83   62.7  003_1  20170302  0935  
1       0  0       0      1    83   62.7  003_2  20170302  0939  
2       0  0       0      1    80   52.6  004_1  20170303  0817  
3       0  0       0      1    80   52.6  004_2  20170303  0827  
4       0  0       0      0    91   68.5  005_1  20170303  0837  


In [36]:
'str'.capitalize()

'Str'

In [37]:
d.columns = [col.capitalize() for col in d.columns]

In [40]:
d[['Id','Date','Time','Age','Gender','Height','Weight','Hr','G']].to_csv('/home/cmchang/BG/meta.csv',index=False)

In [50]:
data_path = '/data/put_data/medical/BG/pro_data_AC'
for sid in d.Id:
    tmp = pd.read_csv(os.path.join(data_path, sid+'.csv'))
    tmp.columns = ['MilliSecond', 'ECG', 'LR', 'LIR', 'RR','RIR']
    tmp[['MilliSecond','ECG']].to_csv(os.path.join('/home/cmchang/BG/data/ECG',sid+'.csv'), index=False)
    print(sid, end='\r')

In [14]:
d.shape

(2238, 20)

# Splitting data

In [29]:
'''split training and testing data set'''
D_id = list(set([row['Person No'] for index, row in d.iterrows() if float(row['G']) >= 200]))
np.random.shuffle(D_id)
ND_id = np.asarray(d.loc[np.asarray(d['G'], dtype = 'float64') < 200]['Person No'].unique())
np.random.shuffle(ND_id)

train_id = np.hstack((ND_id[:int(len(ND_id) * split_ratio)], D_id[:int(len(D_id) * split_ratio)]))
# train_id = np.hstack((ND_id[:int(len(ND_id) * split_ratio)]))
train_id

test_id = np.hstack((ND_id[int(len(ND_id) * split_ratio):], D_id[int(len(D_id) * split_ratio):]))
# test_id = np.hstack((ND_id[int(len(ND_id) * split_ratio):]))
test_id

test_ind = [index for index, row in d.iterrows() if np.any(test_id == row['Person No'])] 
np.random.shuffle(test_ind)
train_ind = [index for index, row in d.iterrows() if np.any(train_id == row['Person No'])]   
np.random.shuffle(train_ind)

train_file = np.array(d['ID'][train_ind])
test_file = np.array(d['ID'][test_ind])
lab = np.array(d['G']).astype('float32')

# train_lab = np.array(lab[train_ind])
# test_lab  = np.array(lab[test_ind])

train_lab = {f:d[d['ID'] == f]['G'].astype('float32').values for f in train_file}
test_lab = {f:d[d['ID'] == f]['G'].astype('float32').values for f in test_file}
print('counts of training: ', train_file.shape[0], ' and counts of testing: ', test_file.shape[0])

counts of training:  1790  and counts of testing:  450


# Model function define

In [30]:
def ResidualBlock(filters,kernel_size,strides,pool_size,inputs):
    if activation == 'selu':
        new_input = MaxPool1D(pool_size=pool_size, padding = 'same', strides = strides)(inputs)
        new_inp_2 = Activation(activation='selu')(inputs)
        new_inp_2 = Dropout(dropout_rate)(new_inp_2)
        new_inp_2 = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(new_inp_2)
        new_inp_2 = Activation(activation='selu')(new_inp_2)
        new_inp_2 = Dropout(dropout_rate)(new_inp_2)
        new_inp_2 = Conv1D(filters = filters, kernel_size = kernel_size, strides = 1, padding = 'same',kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(new_inp_2)
        new_inp_2 = Add()([new_inp_2, new_input])
        return new_inp_2
    else:
        new_input = MaxPool1D(pool_size=pool_size, padding = 'same', strides = strides)(inputs)
        new_inp_2 = BatchNormalization()(inputs)
        new_inp_2 = Activation(activation=activation)(new_inp_2)
        new_inp_2 = Dropout(dropout_rate)(new_inp_2)
        new_inp_2 = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(new_inp_2)
        new_inp_2 = BatchNormalization()(new_inp_2)
        new_inp_2 = Activation(activation=activation)(new_inp_2)
        new_inp_2 = Dropout(dropout_rate)(new_inp_2)
        new_inp_2 = Conv1D(filters = filters, kernel_size = kernel_size, strides = 1, padding = 'same',kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(new_inp_2)
        new_inp_2 = Add()([new_inp_2, new_input])
        return new_inp_2

In [31]:
def local_cnn(filters,kernel_size,strides,pool_size,inputs):
    if activation == 'selu':
        inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',kernel_initializer = kernel_initializer)(inputs)
        inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',kernel_initializer = kernel_initializer)(inp)
        inp = MaxPool1D(pool_size = local_pool_size, padding = 'same')(inp)
        inp = Activation(activation='selu')(inp)
        inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',kernel_initializer = kernel_initializer)(inp)
        inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',kernel_initializer = kernel_initializer)(inp)
    else:
        inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',kernel_initializer = kernel_initializer)(inputs)
        inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',kernel_initializer = kernel_initializer)(inp)
        inp = MaxPool1D(pool_size = local_pool_size, padding = 'same')(inp)
        inp = BatchNormalization()(inp)
        inp = Activation(activation=activation)(inp)
        inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',kernel_initializer = kernel_initializer)(inp)
        inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',kernel_initializer = kernel_initializer)(inp)
    return inp

# Build model

In [32]:
def build_model():
    global strides, filters
    print('Use multi CNN: ', multi_CNN, ' activation function: ', activation, '\n')
    print('Start building model.....')
    if use_meta:
        meta_inp = Input(shape=(n_meta,))
    if multi_CNN:
        sig_inp5 = Input(shape = (3000, 10))
        inp5 = local_cnn(local_filters,local_kernel_size,strides,local_pool_size,sig_inp5)

        sig_inp10 = Input(shape = (6000, 10))
        inp10 = local_cnn(local_filters, local_kernel_size, strides, local_pool_size, sig_inp10)

        sig_inp25 = Input(shape = (750, 10))
        inp25 = local_cnn(local_filters, local_kernel_size, strides, local_pool_size, sig_inp25)

        sig_inp50 = Input(shape = (1500, 10))
        inp50 = local_cnn(local_filters, local_kernel_size, strides, local_pool_size, sig_inp50)
        if activation == 'selu':
            inp_con = keras.layers.concatenate([inp5, inp10, inp25, inp50], axis = 1)
            inp_max = MaxPool1D(pool_size=2, padding = 'same')(inp_con)
            l1 = Conv1D(filters=filters,kernel_size=kernel_size,strides=2,padding="same",kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(inp_con)
            l1 = Activation(activation='selu')(l1)
            l1 = Dropout(dropout_rate)(l1)
            l1 = Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same",kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(l1)
            new_inp = Add()([l1,inp_max])

            for i in range(layers):
            # every alternate residual block subsample its input by a factor of 2
                if i % 2 == 1:
                    pool_size = 2
                    strides = 2
                else:
                    pool_size = 1
                    strides = 1
                # incremented filters    
                if i % 4 == 3:
                    filters = 64*int(i//4 + 2)
                    new_inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = 1, padding = 'same',kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(new_inp)
                new_inp = ResidualBlock(filters,kernel_size,strides,pool_size,new_inp)

            new_inp = GlobalAveragePooling1D()(new_inp)
            new_inp = Activation(activation='selu')(new_inp)
            if use_meta:
                new_inp = keras.layers.concatenate([new_inp,meta_inp])
            new_inp = Dense(64,activation='selu', kernel_regularizer=regularizers.l2(l_2))(new_inp)
            new_inp = Dropout(dropout_rate)(new_inp)
            out = Dense(1,activation='linear', kernel_regularizer=regularizers.l2(l_2))(new_inp)
        else:
            inp_con = keras.layers.concatenate([inp5, inp10, inp25, inp50], axis = 1)
            inp_max = MaxPool1D(pool_size=2, padding = 'same')(inp_con)
            l1 = Conv1D(filters=filters,kernel_size=kernel_size,strides=2,padding="same",kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(inp_con)
            l1 = BatchNormalization()(l1)
            l1 = Activation(activation=activation)(l1)
            l1 = Dropout(dropout_rate)(l1)
            l1 = Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same",kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(l1)
            new_inp = Add()([l1,inp_max])

            for i in range(layers):
            # every alternate residual block subsample its input by a factor of 2
                if i % 2 == 1:
                    pool_size = 2
                    strides = 2
                else:
                    pool_size = 1
                    strides = 1
                # incremented filters    
                if i % 4 == 3:
                    filters = 64*int(i//4 + 2)
                    new_inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = 1, padding = 'same',kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(new_inp)
                new_inp = ResidualBlock(filters,kernel_size,strides,pool_size,new_inp)
#             new_inp = Flatten()(new_inp)
            new_inp = GlobalAveragePooling1D()(new_inp)
            new_inp = BatchNormalization()(new_inp)
            new_inp = Activation(activation=activation)(new_inp)
#             new_inp = Dense(192,kernel_regularizer=regularizers.l2(l_2),kernel_initializer = kernel_initializer)(new_inp)
            if use_meta:
                new_inp = keras.layers.concatenate([new_inp,meta_inp])
#             new_inp = BatchNormalization()(new_inp)
#             new_inp = Activation(activation=activation)(new_inp)
#             new_inp = Dropout(dropout_rate)(new_inp)
#             new_inp = Dense(128,kernel_regularizer=regularizers.l2(l_2),kernel_initializer = kernel_initializer)(new_inp)
#             new_inp = BatchNormalization()(new_inp)
#             new_inp = Activation(activation=activation)(new_inp)
#             new_inp = Dropout(dropout_rate)(new_inp)
#             new_inp = Dense(128,kernel_regularizer=regularizers.l2(l_2),kernel_initializer = kernel_initializer)(new_inp)
#             new_inp = BatchNormalization()(new_inp)
#             new_inp = Activation(activation=activation)(new_inp)
#             new_inp = Dropout(dropout_rate)(new_inp)
            new_inp = Dense(128,activation=activation, kernel_regularizer=regularizers.l2(l_2))(new_inp)
            new_inp = Dropout(dropout_rate)(new_inp)
            new_inp = Dense(128,activation=activation, kernel_regularizer=regularizers.l2(l_2))(new_inp)
            #new_inp = keras.layers.concatenate([new_inp,meta_inp])
            out = Dense(1,activation='linear', kernel_regularizer=regularizers.l2(l_2))(new_inp)
        if use_meta:
            model = Model(inputs=[sig_inp5, sig_inp10, sig_inp25, sig_inp50,meta_inp],outputs=[out])
        else:
            model = Model(inputs=[sig_inp5, sig_inp10, sig_inp25, sig_inp50],outputs=[out])
    else:
        sig_inp =  Input(shape=(n_sample, n_channel))        
        if activation == 'selu':
            inp = Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same")(sig_inp)
            inp = Activation(activation='selu')(inp)
            inp_max = MaxPool1D(pool_size=2)(inp)

            l1 = Conv1D(filters=filters,kernel_size=kernel_size,strides=2,padding="same")(inp)
            l1 = Activation(activation='selu')(l1)
            l1 = Dropout(dropout_rate)(l1)
            l1 = Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same")(l1)
            new_inp = Add()([l1,inp_max])

            for i in range(layers):
            # every alternate residual block subsample its input by a factor of 2
                if i % 2 == 1:
                    pool_size = 2
                    strides = 2
                else:
                    pool_size = 1
                    strides = 1
                # incremented filters    
                if i % 4 == 3:
                    filters = 64*int(i//4 + 2)
                    new_inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = 1, padding = 'same',kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(new_inp)
                new_inp = ResidualBlock(filters,kernel_size,strides,pool_size,new_inp)

            new_inp = GlobalAveragePooling1D()(new_inp)
            new_inp = Activation(activation='selu')(new_inp)
            if use_meta:
                new_inp = keras.layers.concatenate([new_inp,meta_inp])
            new_inp = Dense(256,activation='selu', kernel_regularizer=regularizers.l2(l_2))(new_inp)
            new_inp = Dropout(dropout_rate)(new_inp)
            out = Dense(1,activation='linear', kernel_regularizer=regularizers.l2(l_2))(new_inp)
        else:
            inp = Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same")(sig_inp)
            inp = BatchNormalization()(inp)
            inp = Activation(activation=activation)(inp)
            inp_max = MaxPool1D(pool_size=2)(inp)

            l1 = Conv1D(filters=filters,kernel_size=kernel_size,strides=2,padding="same")(inp)
            l1 = BatchNormalization()(l1)
            l1 = Activation(activation=activation)(l1)
            l1 = Dropout(dropout_rate)(l1)
            l1 = Conv1D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same")(l1)

            new_inp = Add()([l1,inp_max])

            for i in range(layers):
            # every alternate residual block subsample its input by a factor of 2
                if i % 2 == 1:
                    pool_size = 2
                    strides = 2
                else:
                    pool_size = 1
                    strides = 1
                # incremented filters    
                if i % 4 == 3:
                    filters = 64*int(i//4 + 2)
                    new_inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = 1, padding = 'same',kernel_initializer = kernel_initializer, kernel_regularizer=regularizers.l2(l_2))(new_inp)
                new_inp = ResidualBlock(filters,kernel_size,strides,pool_size,new_inp)

            new_inp = GlobalAveragePooling1D()(new_inp)
            new_inp = BatchNormalization()(new_inp)
            new_inp = Activation(activation=activation)(new_inp)
            if use_meta:
                new_inp = keras.layers.concatenate([new_inp,meta_inp])
            new_inp = Dense(128,activation=activation, kernel_regularizer=regularizers.l2(l_2))(new_inp)
            new_inp = Dropout(dropout_rate)(new_inp)
            new_inp = Dense(128,activation=activation, kernel_regularizer=regularizers.l2(l_2))(new_inp)
            #new_inp = keras.layers.concatenate([new_inp,meta_inp])
            out = Dense(1,activation='linear', kernel_regularizer=regularizers.l2(l_2))(new_inp)
        if use_meta:
            model = Model(inputs=[sig_inp,meta_inp],outputs=[out])
        else:
            model = Model(inputs=[sig_inp],outputs=[out])
    return model

# Model compile

In [33]:
model = build_model()
adam = Adam(lr=lr_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay = 0.0)
model.compile(loss='mae', optimizer=adam)
model.summary()

Use multi CNN:  False  activation function:  relu 

Start building model.....
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 6000, 10)      0                                            
____________________________________________________________________________________________________
conv1d_51 (Conv1D)               (None, 6000, 64)      10304       input_6[0][0]                    
____________________________________________________________________________________________________
batch_normalization_24 (BatchNor (None, 6000, 64)      256         conv1d_51[0][0]                  
____________________________________________________________________________________________________
activation_47 (Activation)       (None, 6000, 64)      0           batch_normalization_24[0][0]     
_____________

# Directory and Model name

In [34]:
'''model name'''
time_str = time.strftime("%m%d")
drop = str(math.ceil(dropout_rate/0.1))
model_name =  'D' + drop + '_l' + str(l_2) +'_multicnn='+str(multi_CNN)+'_L'+str(layers*2+3)+'_GVP_Meta=' + str(use_meta) + '_kernal=' + str(kernel_size)
directory = 'train_' + time_str + '/' + model_name + '/' 

'''model training'''
print('Model will be saved to ', directory, '...........')

if not os.path.exists(directory):
    os.makedirs(directory)

Model will be saved to  train_1012/D2_l0_multicnn=False_L23_GVP_Meta=True_kernal=16/ ...........


# Training tools

In [35]:
checkpoint = ModelCheckpoint(directory + "model.h5",
                    monitor='val_loss',
                    verbose=1,
                    save_best_only=True,
                    mode='auto')

def figure_making(train_loss, val_loss):
    sk = 0
    plt.figure(0)
    plt.plot(range(len(train_loss[sk:])),train_loss[sk:],label='train_loss')
    plt.plot(range(len(val_loss[sk:])),val_loss[sk:],label='val_loss')
    plt.legend(loc='upper right')
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    plt.ylim([0, max(train_loss+val_loss)])
    plt.savefig(directory + "loss_history.png", dpi = 300)
    plt.close()

class LossHistory(Callback):
    def on_train_begin(self,logs={}):
        self.loss=[]
        self.val_loss=[]
    def on_epoch_end(self,epoch,logs={}):
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
        if len(self.loss) % 30 == 0 and len(self.loss) != 0:
            figure_making(self.loss, self.val_loss)

loss_history = LossHistory()

earlystopping = EarlyStopping(monitor='val_loss', patience = 30, mode = 'auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.3, patience = 5, min_lr = 0, cooldown = 5, verbose = True)

# Model training

In [36]:
if multi_CNN:
    if use_meta:
        Val_X5, Val_X10, Val
        _X25, Val_X50, Val_M,Val_Y = validation_generation(test_file,test_lab,multi_CNN,use_meta,X5,X10,X25,X50,meta)
        model.fit_generator(generator=train_generator(train_file,batch_size,multi_CNN,use_meta,X5,X10,X25,X50,meta,train_lab),
                        validation_data=([Val_X5, Val_X10, Val_X25,Val_X50,Val_M],Val_Y),
                        steps_per_epoch = n_batch_per_epoch,
                        epochs=epochs,
                        callbacks = [loss_history,checkpoint,earlystopping,reduce_lr])
    else:
        Val_X5, Val_X10, Val_X25, Val_X50,Val_Y = validation_generation(test_file,test_lab,multi_CNN,use_meta,X5,X10,X25,X50,meta)
        model.fit_generator(generator=train_generator(train_file,batch_size,multi_CNN,use_meta,X5,X10,X25,X50,meta,train_lab),
                                validation_data=([Val_X5, Val_X10, Val_X25,Val_X50],Val_Y),
                                steps_per_epoch = n_batch_per_epoch,
                                epochs=epochs,
                                callbacks = [loss_history,checkpoint,earlystopping,reduce_lr])
else:
    if use_meta:
        Val_X, Val_M,Val_Y = validation_generation_single(test_file,test_lab,X,meta,use_meta)
        model.fit_generator(generator=train_generator_single(train_file, batch_size, X, meta, train_lab, use_meta),
                        validation_data=([Val_X,Val_M],Val_Y),
                        steps_per_epoch = n_batch_per_epoch,
                        epochs=epochs,
                        callbacks = [loss_history,checkpoint,earlystopping,reduce_lr])
    else:
        Val_X50,Val_Y = validation_generation_single(test_file,test_lab,X,meta,use_meta)
        model.fit_generator(generator=train_generator_single(train_file, batch_size, X, meta, train_lab, use_meta),
                            validation_data=([Val_X50],Val_Y),
                            steps_per_epoch = n_batch_per_epoch,
                            epochs=epochs,
                            callbacks = [loss_history,checkpoint,earlystopping,reduce_lr])

# records =  2250
Epoch 1/200
200/200 [==============================] - 118s - loss: 106.8069 - val_loss: 103.6567
Epoch 2/200
200/200 [==============================] - 77s - loss: 104.5313 - val_loss: 100.5003
Epoch 3/200
200/200 [==============================] - 77s - loss: 102.0654 - val_loss: 97.6195
Epoch 4/200
200/200 [==============================] - 77s - loss: 98.8383 - val_loss: 94.7820
Epoch 5/200
200/200 [==============================] - 77s - loss: 94.9525 - val_loss: 90.6360
Epoch 6/200
200/200 [==============================] - 77s - loss: 89.8890 - val_loss: 85.1775
Epoch 7/200
200/200 [==============================] - 77s - loss: 83.1568 - val_loss: 77.5637
Epoch 8/200
200/200 [==============================] - 77s - loss: 74.3349 - val_loss: 68.8643
Epoch 9/200
200/200 [==============================] - 77s - loss: 62.8709 - val_loss: 56.9167
Epoch 10/200
200/200 [==============================] - 77s - loss: 49.6039 - val_loss: 42.9166
Epoch 11/200
200/200 [===

KeyboardInterrupt: 

# Plotting and Saving DataFrame

In [38]:
'''loss history fig'''
import matplotlib.pyplot as plt
matplotlib.use('Agg')
sk = 0
plt.figure(0)
plt.plot(range(len(loss_history.loss[sk:])),loss_history.loss[sk:],label='train_loss')
plt.plot(range(len(loss_history.val_loss[sk:])),loss_history.val_loss[sk:],label='val_loss')
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.ylim([0, max(loss_history.val_loss+loss_history.loss)])
plt.savefig(directory + "loss_history.png", dpi = 300)
plt.close()

'''plot train/test mae/cor fig'''
tmp = [model_name]

# recording
tmp.append(loss_history.loss[loss_history.val_loss.index(min(loss_history.val_loss))])
tmp.append(min(loss_history.val_loss))
tmp.append(loss_history.loss[-1])
tmp.append(loss_history.val_loss[-1])

model_test = load_model(directory + "model.h5")

# plot training
if use_meta and multi_CNN:
    train_X1, train_X2, train_X3, train_X4, train_M,train_Y1 = one_train_set(multi_CNN,use_meta,X5,X10,X25,X50,meta,train_lab,train_file=train_file)
    Y_pred = model_test.predict([train_X1, train_X2, train_X3, train_X4,train_M])
else:
    train_X1, train_meta, train_Y1 = one_train_set_single(X,meta,train_lab,train_file,use_meta,size = 2000)
    Y_pred = model_test.predict([train_X1,train_meta])
Y_pred = np.hstack(Y_pred)

cor = pearsonr(train_Y1,Y_pred)[0]
mae = mean_absolute_error(train_Y1,Y_pred)

# recording
tmp.append(mae)
tmp.append(cor)


f = pd.DataFrame({'y_true': train_Y1,'y_pred': Y_pred})
f.to_csv(directory + "pred_train.csv")


plt.figure(0)
plt.scatter(train_Y1, Y_pred, alpha = .15, s = 20)
plt.xlabel('True_Y')
plt.ylabel('Pred_Y')
plt.title("Training data \n" + "MAE = %4f; Cor = %4f; #samples = %d" % (mae,cor, len(train_Y1)))
plt.savefig(directory+"plot_scatter_train.png", dpi = 300)
plt.close()

# plot testing
if use_meta and multi_CNN:
    Y_pred = model_test.predict([Val_X5, Val_X10, Val_X25, Val_X50, Val_M])
else:
    Y_pred = model_test.predict([Val_X, Val_M])
    
Y_pred = np.hstack(Y_pred)

f = pd.DataFrame({'y_true': Val_Y,'y_pred': Y_pred})
f.to_csv(directory + "pred_test.csv")

cor = pearsonr(Val_Y,Y_pred)[0]
mae = mean_absolute_error(Val_Y,Y_pred)

# recording
tmp.append(mae)
tmp.append(cor)

'''scatter plot for test data'''
plt.figure(0)
plt.scatter(Val_Y, Y_pred, alpha = .15, s = 20)
plt.xlabel('True_Y')
plt.ylabel('Pred_Y')
plt.title("Testing data \n" + "MAE = %4f; Cor = %4f; #samples = %d" % (mae, cor, len(Val_Y)))
plt.savefig(directory + "plot_scatter_test.png", dpi = 300)
plt.close()

'''save model detail'''
f = pd.DataFrame([tmp], columns = ['model_name', 'train_loss', 'val_loss', 'final_train_loss', 'final_val_loss', 'train_mae', 'train_cor', 'test_mae', 'test_cor'])
f.to_csv(directory + 'model_detail.csv')

'''save test ID'''
f2 = pd.DataFrame([ID for ID in test_file], columns = ['test_ID'])
f2.to_csv(directory + 'test_id.csv')

/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# Output Flatten Layers

In [6]:
directory = 'train_1008/D0_l0_multicnn=False_L23_GVP_Meta=True_kernal=32/'
model = load_model(directory+'model.h5')
model.layers
new_model = Model(inputs=[model.layers[0].input], outputs=model.layers[-9].output)
# new_model = Model(inputs=[model.layers[0].input,model.layers[1].input,model.layers[2].input,model.layers[3].input], outputs=model.layers[-11].output)
new_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 6000, 10)      0                                            
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 6000, 64)      20544       input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 6000, 64)      256         conv1d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 6000, 64)      0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [16]:
# train_file = np.array(d['ID'][:len(d)])
# train_lab = np.array(d['G'][:len(d)])
# train_lab = train_lab.astype('float32')

# def validation_generation(test_file, test_lab):
#     X5_ = []
#     X10_ = []
#     X25_ = []
#     X50_ = []
#     Y_ = []
#     ind_ = []
#     for i in range(test_file.shape[0]):
#         x = X5[test_file[i]] 
#         x10 = X10[test_file[i]]
#         x25 = X25[test_file[i]]
#         x50 = X50[test_file[i]]
#         X5_.append(x[random.randint(0, 4)])
#         X10_.append(x10[random.randint(0, 4)])
#         X25_.append(x25[random.randint(0, 4)])
#         X50_.append(x50[random.randint(0, 4)])
#         Y_.append(test_lab[i])
#         ind_.append(test_file[i])
#     X5_ = np.asarray(X5_)
#     X10_ = np.asarray(X10_)
#     X25_ = np.asarray(X25_)
#     X50_ = np.asarray(X50_)
#     Y_ = np.asarray(Y_)
#     return X5_, X10_,X25_,X50_, Y_, ind_

# X5_,X10_,X25_,X50_,Y_,ind_ = validation_generation(train_file,train_lab)
# t = new_model.predict([X5_,X10_,X25_,X50_])
# ind_ = pd.DataFrame(ind_, columns = ['id'])
# t1 = pd.DataFrame(t, dtype = 'str')
# t2 = pd.concat([ind_, t1], axis = 1)
# t2.to_csv(directory+'flatten.csv')
# t2.shape

# Release GPU Memory

In [17]:
#%%javascript
# Jupyter.notebook.session.delete();